# OpenUVF Model Trainer

This notebook provides an interface to simplify and unify the training process of further models. The notebook allows the user to specify the directories of their inputs and the desired directories of their outputs then generates t

In [31]:
#Import Essential Packages
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
import os
import io
import random
import glob
import logging
import subprocess
import shutil
from PIL import Image
from collections import namedtuple, OrderedDict

#Import Tensorflow Utilities
import tensorflow as tf
from core import object_detection
from core import nets
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util
from object_detection import model_main
from object_detection import model_hparams
from object_detection import model_lib

#Import OpenUVF Utilities
from core.utils.create_tfrecord import create_tf_record
from core.utils.augment_images import augment_images

%matplotlib inline

## Input Directories

###### NOTE: When specifying absolute paths, the drive must be of the format "C:\\" or "C:/". C:\, as copied from windows explorer, will throw an error.

Parameters:
  1. images_dir (str) = path to images used for trainng
  2. labels_dir (str) = path to label map
  3. annotations_in_dir (str) = path to PASCAL VOC annotation xmls
  4. annotations_out_dir (str) = path to output TFRecords
  5. 

In [32]:
#Input Directories - Provide as full path (e.g. C:/)
images_dir = 'sets/M1C1/images/augmented'
labels_dir = 'core/labels/1_Class_label_map.pbtxt'
annotations_in_dir = images_dir
annotations_out_dir = 'sets/M1C1/tfrecords'
pipeline_config_path = 'core/models/M1C1(FRCNN_RN101_COCO)_v1/pipeline.config'
training_dir = 'workspace/M1C1(FRCNN_RN101_COCO)/training'
pre_trained_model_dir = 'core/models/M1C1(FRCNN_RN101_COCO)_v1'

## Input Settings 

##### Function Settings
  1. output_image_sets
  2. augment_image_sets
  3. built_TFRecord
  4. execute_train
  5. execute_eval
  6. open_tensorboard
  7. image_ext
  8. ignore_difficult
  9. func_debug
  
##### Training Settings  
  1. train_frac (float) = fraction of images + annotations to use for training. 1-train_frac is the amount used for testing.
  2. num_shards (int) = number of output sharded (divided) TFRecords. Increase this as training sets become too large.
  3. pipeline_dir (str) = path to pipeline.config file
  4. training_dir (str) = path to folder where training files (checkpoints) should be stored
  5. num_train_steps (int) = number of training steps (how many steps the trainer goes through before automatically exiting)
  6. num_eval_steps (int) = number of evaluation steps
  
##### Augmentation Settings  

In [33]:
#Function Settings
output_image_sets = True
augment_image_sets = False
build_TFRecords = False
execute_train = True
execute_eval = True
open_tensorboard = True
update_pipeline = True
image_ext = '.PNG'
ignore_difficult = False
func_debug = True
save_train_and_test_images = True

#Training Settings
train_frac = .8
isolate_sets = True
num_shards = 1
num_train_steps = 200000
num_test_steps = 20000

#Augmentation Settings
augment_rotations = [0, 180,  0,  180]
augment_mirrored = [False, False, True, True]
augmentations_per_image = 4



## Input Parsing

In [34]:



#Ensuring all input paths are absolute
if not os.path.isabs(images_dir):
    images_dir = os.path.join(os.getcwd(), images_dir)
#if not os.path.isabs(labels_dir):
 #   labels_dir = os.path.join(os.getcwd(), labels_dir)
if not os.path.isabs(annotations_in_dir):
    annotations_in_dir = os.path.join(os.getcwd(), annotations_in_dir)
#if not os.path.isabs(annotations_out_dir):
#    annotations_out_dir = os.path.join(os.getcwd(), annotations_out_dir)
if not os.path.isabs(pipeline_config_path):
    pipeline_config_path = os.path.join(os.getcwd(), pipeline_config_path)
#if not os.path.isabs(training_dir):
#    training_dir = os.path.join(os.getcwd(), training_dir)
    
#Checking if input images and annotations are in the same folder
images_and_annotations_together = images_dir == annotations_in_dir

#Defining Additional Paths
train_output_path = annotations_out_dir + '/train.record'
eval_output_path = annotations_out_dir + '/eval.record'
fine_tune_checkpoint_path = os.path.join(pre_trained_model_dir, 'model.ckpt')

## Image Set Refactoring

## Image Augmentation

In [35]:
#Image Augmentation
if augment_image_sets:

    #Call Augmentation Function    
    augment_images(images_dir, annotations_in_dir, image_ext, augment_rotations, augment_mirrored, ignore_difficult)

    
    
    #Update images_dir to augmented folder
    images_dir = images_dir + '/augmented'
    
    #TEMPORARY - REMOVE
    annotations_in_dir = images_dir 
    
    #Print Feedback to User
    logging.info('Images Augmented. Augmented Images saved to: ' + images_dir)
    


# Generate TFRecords for Training and Testing Sets

From input images and corresponding xmls, this section produces the TFRecord files required to train and evaluate the model.




=================================================================================================================
Code derived from create_pet_tf_record.py by The TensorFlow Authors. Available unmodified at: https://github.com/tensorflow/models/blob/master/research/object_detection/dataset_tools/create_pet_tf_record.py

 Copyright 2017 The TensorFlow Authors. All Rights Reserved.

 Licensed under the Apache License, Version 2.0 (the "License");
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.
 
=================================================================================================================

In [36]:
if build_TFRecords:
    
    #PROBLEM: NEED IT TO OUTPUT FILENAME AS WELL. SHARDING CAUSES DISCREPANCY AND ERROR
    
    #Divide Image Set into Training and Evaluation Sets
    label_map_dict = label_map_util.get_label_map_dict(labels_dir)
    examples_list = glob.glob(images_dir + '/*.xml')
    examples_list = [os.path.splitext(os.path.split(example)[1])[0] for example in examples_list]
    if isolate_sets:

        #Check if set is augmented - augmented if first image has -A0 as its terminal block in name
        name = os.path.splitext(examples_list[0])[0]
        augmented = name.split("-")[-1] == 'A0'

        #Iterate through examples_list and identify base names
        base_examples_list = []
        auglabels = []
        for example in examples_list:
            #Remove extension
            name = os.path.splitext(example)[0]

            #Remove augmentation label
            name = name.rpartition('-')
            auglabel = name[-1]
            name = name[0]

            #log the example if it is the first of its kind, otherwise ignore
            if not name in base_examples_list:
                base_examples_list.append(name)

            #log auglabel if it is the first of its kind
            if not auglabel in auglabels:
                auglabels.append(auglabel)

        #Randomly allocate base image set to Training and Evaluation Sets
        random.seed(42)
        random.shuffle(base_examples_list)
        num_examples = len(base_examples_list)
        num_train = int(train_frac * num_examples)
        base_train_examples = base_examples_list[:num_train]
        base_eval_examples = base_examples_list[num_train:]

        #Iterate through base examples and assign to actual list
        train_examples = []
        eval_examples = []
        for example in base_train_examples:
            for auglabel in auglabels:
                train_examples.append(example + '-' + auglabel)

        for example in base_eval_examples:
            for auglabel in auglabels:
                eval_examples.append(example + '-' + auglabel)

    else:

        #Randomly Dividing the input set into training and testing sets
        random.seed(42)
        random.shuffle(examples_list)
        num_examples = len(examples_list)
        num_train = int(train_frac * num_examples)
        train_examples = examples_list[:num_train]
        eval_examples = examples_list[num_train:]
        logging.info('%d training and %d testing examples.',
                       len(train_examples), len(eval_examples))

    #Saving training and testing examples if requested
    if save_train_and_test_images:

        #Creating Output Directories
        train_out_dir = os.path.join(images_dir, 'training')
        test_out_dir = os.path.join(images_dir, 'evaluation')
        if not os.path.isdir(train_out_dir):
            os.mkdir(train_out_dir)
        if not os.path.isdir(test_out_dir):
            os.mkdir(test_out_dir)

        for ex in train_examples:
            imname = ex + image_ext
            im = Image.open(os.path.join(images_dir, imname))
            im.save(os.path.join(train_out_dir, imname))

        for ex in eval_examples:
            imname = ex + image_ext
            im = Image.open(os.path.join(images_dir, imname))
            im.save(os.path.join(test_out_dir, imname))


    #Passing inputs to function
    create_tf_record(
        train_output_path,
        num_shards,
        label_map_dict,
        annotations_in_dir,
        images_dir,
        train_examples,
        faces_only=True,
        mask_type='png')
    create_tf_record(
        eval_output_path,
        num_shards,
        label_map_dict,
        annotations_in_dir,
        images_dir,
        eval_examples,
        faces_only=True,
        mask_type='png')

## Automatically Updating Pipeline config

In [37]:
if update_pipeline:
    
    #Define additional (optional) inputs
    
    
    #Load/Create files
    pipeline_in = open(pipeline_config_path)
    pipeline_out = open(os.path.join(training_dir, 'pipeline.config'), 'wt')
    pipeline_out_backup = open(os.path.join(training_dir, 'pipeline_backup.config'), 'wt')

    #Iterate line by line and update paths
    train_current = False
    eval_current = False
    for line in pipeline_in:
        
        #Check for header keywords
        if 'train_input_reader' in line:
            train_current = True
            eval_current = False
        elif 'eval_input_reader' in line:
            eval_current = True
            train_current = False
        
        #Check for line labels that need replacement
        if 'label_map_path' in line:
            elements = line.rpartition(':')
            line = elements[0] + ': "' + labels_dir + '"\n'
        elif 'input_path' in line and train_current:
            elements = line.rpartition(':')
            line = elements[0] + ': "' + train_output_path + '"\n'
        elif 'input_path' in line and eval_current:
            elements = line.rpartition(':')
            line = elements[0] + ': "' + eval_output_path + '"\n'
        elif 'fine_tune_checkpoint' in line:
            elements = line.rpartition(':')
            line = elements[0] + ': "' + fine_tune_checkpoint_path + '"\n'
            
        #Writing lines to files    
        pipeline_out.write(line)
        pipeline_out_backup.write(line)
        print(line)

        
    #Close files
    pipeline_in.close()
    pipeline_out.close()
    pipeline_out_backup.close()
    
    #Define new pipeline path - move to training folder so it is local to specific instance
    pipeline_config_path = os.path.join(training_dir, 'pipeline.config')
    

model {

  faster_rcnn {

    num_classes: 1

    image_resizer {

      keep_aspect_ratio_resizer {

        min_dimension: 800

        max_dimension: 1700

      }

    }

    feature_extractor {

      type: "faster_rcnn_resnet101"

      first_stage_features_stride: 16

    }

    first_stage_anchor_generator {

      grid_anchor_generator {

        height_stride: 16

        width_stride: 16

        scales: 0.25

        scales: 0.5

        scales: 1.0

        scales: 2.0

        aspect_ratios: 0.5

        aspect_ratios: 1.0

        aspect_ratios: 2.0

      }

    }

    first_stage_box_predictor_conv_hyperparams {

      op: CONV

      regularizer {

        l2_regularizer {

          weight: 0.0

        }

      }

      initializer {

        truncated_normal_initializer {

          stddev: 0.009999999776482582

        }

      }

    }

    first_stage_nms_score_threshold: 0.0

    first_stage_nms_iou_threshold: 0.699999988079071

    first_stage_max_proposals: 3

## Run Train + Evaluation

Code derived from model_main.py by The TensorFlow Authors

In [40]:
if execute_train & execute_eval:

    #Define Variables
    hparams_overrides = None
    sample_1_of_n_eval_examples = 1
    sample_1_of_n_eval_on_train_examples = 5
    checkpoint_dir = None
    run_once = False
    eval_training_data = False
    train_steps = num_train_steps
    
    #Run Estimator Config
    config = tf.estimator.RunConfig(model_dir=training_dir)

    #Define Training and Evaluation Dict
    train_and_eval_dict = model_lib.create_estimator_and_inputs(
        run_config=config,
        hparams=model_hparams.create_hparams(hparams_overrides),
        pipeline_config_path=pipeline_config_path,
        train_steps=train_steps,
        sample_1_of_n_eval_examples=sample_1_of_n_eval_examples,
        sample_1_of_n_eval_on_train_examples=(
            sample_1_of_n_eval_on_train_examples))
    estimator = train_and_eval_dict['estimator']
    train_input_fn = train_and_eval_dict['train_input_fn']
    eval_input_fns = train_and_eval_dict['eval_input_fns']
    eval_on_train_input_fn = train_and_eval_dict['eval_on_train_input_fn']
    predict_input_fn = train_and_eval_dict['predict_input_fn']
    train_steps = train_and_eval_dict['train_steps']

    if checkpoint_dir:
        
        if eval_training_data:
            name = 'training_data'
            input_fn = eval_on_train_input_fn
        else:
            name = 'validation_data'
            # The first eval input will be evaluated.
            input_fn = eval_input_fns[0]
            
        if run_once:
            estimator.evaluate(input_fn,
                               num_eval_steps=None,
                               checkpoint_path=tf.train.latest_checkpoint(
                                   checkpoint_dir))
            
        else:
            model_lib.continuous_eval(estimator, checkpoint_dir, input_fn,
                                      train_steps, name)
    else:
        train_spec, eval_specs = model_lib.create_train_and_eval_specs(
            train_input_fn,
            eval_input_fns,
            eval_on_train_input_fn,
            predict_input_fn,
            train_steps,
            eval_on_train_data=False)

        # Currently only a single Eval Spec is allowed.
        tf.estimator.train_and_evaluate(estimator, train_spec, eval_specs[0])

    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
#Calling model_main.py
#args = ["python", "model_main.py", "--pipeline_config_path=", pipeline_config_path, "--model_dir=", 
#        training_dir, "--num_train_steps", num_train_steps, "--num_eval_steps", num_test_steps]
#args = ["python model_main.py --pipeline_config_path=" + pipeline_config_path + " --model_dir=" + 
#        training_dir + " --num_train_steps="+ num_train_steps+ " --num_eval_steps="+ num_test_steps]
#subprocess.call(args, shell=True)

W1204 18:42:50.592568 11724 tf_logging.py:125] Forced number of epochs for all eval validations to be 1.


INFO:tensorflow:Maybe overwriting train_steps: 200000


I1204 18:42:50.593547 11724 tf_logging.py:115] Maybe overwriting train_steps: 200000


INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1


I1204 18:42:50.594550 11724 tf_logging.py:115] Maybe overwriting sample_1_of_n_eval_examples: 1


...

INFO:tensorflow:Done running local_init_op.


I1204 18:53:21.239485 11724 tf_logging.py:115] Done running local_init_op.


creating index...
index created!
INFO:tensorflow:Loading and preparing annotation results...


I1204 18:54:07.844350  7984 tf_logging.py:115] Loading and preparing annotation results...


InvalidArgumentError: TypeError: can't pickle dict_values objects
Traceback (most recent call last):

  File "C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\tensorflow\python\ops\script_ops.py", line 206, in __call__
    ret = func(*args)

  File "C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\object_detection-0.1-py3.6.egg\object_detection\metrics\coco_evaluation.py", line 358, in first_value_func
    self._metrics = self.evaluate()

  File "C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\object_detection-0.1-py3.6.egg\object_detection\metrics\coco_evaluation.py", line 207, in evaluate
    self._detection_boxes_list)

  File "C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\object_detection-0.1-py3.6.egg\object_detection\metrics\coco_tools.py", line 118, in LoadAnnotations
    results.dataset['categories'] = copy.deepcopy(self.dataset['categories'])

  File "C:\Anaconda3\envs\tensorflowGPU\lib\copy.py", line 169, in deepcopy
    rv = reductor(4)

TypeError: can't pickle dict_

In [ ]:
#Open Tensorboard for Review

In [ ]:
#Main Function
def main(_):

    #Generate TFRecord
    TFRecord = dict_to_tf_example()
